# Installing pipeline tools for plasmidome analysis


Environment installations follows
[conda tutorial](https://anaconda.org/anaconda/conda) and [mamba tutorial](https://anaconda.org/conda-forge/mamba) by Anaconda.org

In [ ]:
# Conda envrionment setup
conda install -c conda-forge conda
# for bioconda
conda install -c bioconda-utils

# mamba environment setup
conda install -c conda-forge mamba

plasmidfinder.py setup follows [Bitbucket documentation](https://bitbucket.org/genomicepidemiology/plasmidfinder/src/master/README.md) that include installation steps with plasmidfinder_db and dependencies

In [ ]:
# go to the directory to store plasmidfinder database
cd /mnt/DATAPOOL/mmibstudentnew/

# clone database repository
git clone https://bitbucket.org/genomicepidemiology/plasmidfinder.git
cd plasmidfinder_db
PLASMID_DB=$(pwd)
# install plasmidfinder database with executable kma_index program
python3 INSTALL.py kma_index

# install dependencies
mamba install cgecore
mamba install tabulate

# install kma version 2.8.1
git clone https://bitbucket.org/genomicepidemiology/kma
mamba install kma
# install blast latest version
ftp://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/
mamba install blast

For plasmidome annotation we used Prokka tool. more information can be found through [Torsten Seemann](https://doi.org/10.1093/bioinformatics/btu153) paper

In [ ]:
# install prokka
conda install -c conda-forge -c bioconda -c defaults prokka
# activate prokka_env
conda activate prokka_env

For Pan-Plasmidome investigation, we utilized Panaroo pipeline. More information can be found [here](https://doi.org/10.1186/s13059-020-02090-4)

In [ ]:
# install panaroo
mamba install -c conda-forge -c bioconda -c defaults panaroo

# add cd-hit and MAFFT for building multiple sequence alignments
mamba install cd-hit
mamba install mafft

# install required dependencies
mamba install biopython numpy networkx gffutils edlib joblib tdqm cd-hit
# install optional dependencies
mamba install prank mafft clustal mash

Resistance Gene Identifier (RGI) was utilized to predict the antibiotic resistomes from our RefSeq plasmids that retrieved from NCBI. More information can be found on [CARD database](https://doi.org/10.1093/nar/gkz935)

In [ ]:
# install rgi with its database
mamba install rgi

# download latest AMR reference data from CARD
wget https://card.mcmaster.ca/latest/data
# unarchive downloaded file
tar -xvf data ./card.jsontar -xvf data ./card.json

# load AMR data in working directory > "/mnt/DATAPOOL/mmibstudentnew/dummy/"
rgi load --card_json /mnt/DATAPOOL/mmibstudentnew/dummy/card.json --local
# load AMR data system wide
rgi load --card_json /mnt/DATAPOOL/mmibstudentnew/dummy/card.json

# check database version
# check local
rgi database --version --local
# check system wide
rgi database --version

# clean previous versions
# clean rgi local
rgi clean --local
# clean system wide
rgi clean

# download wildcard data
wget -O wildcard_data.tar.bz2 https://card.mcmaster.ca/latest/variants
mkdir -p wildcard
tar -xjf wildcard_data.tar.bz2 -C wildcard
gunzip wildcard/*.gz

# create annotation files using CARD data
rgi card_annotation -i /mnt/DATAPOOL/mmibstudentnew/dummy/card.json > card_annotation.log 2>&1
# create annotation files using WILDCARD data
rgi wildcard_annotation -i wildcard --card_json /mnt/DATAPOOL/mmibstudentnew/dummy/card.json > wildcard_annotation.log 2>&1

#load data into RGI
rgi load --card_json/mnt/DATAPOOL/mmibstudentnew/dummy/card.json \ 
--debug --local \ 
--card_annotation card_database_v3.2.6.fasta \ 
--card_annotation_all_models card_database_v3.2.6_all.fasta \ 
--wildcard_annotation wildcard_database.fasta \ 
--wildcard_annotation_all_models wildcard_database_all.fasta \ 
--wildcard_index /path/to/wildcard/index-for-model-sequences.txt \ 
--wildcard \ 
--amr_kmers /path/to/wildcard/all_amr_61mers.txt \ 
--kmer_database /path/to/wildcard/61_kmer_db.json \ 
--kmer_size 61

[ABRicate](https://github.com/tseemann/abricate) package used for mass screening of the virulence genes

In [ ]:
# install ABRicate
mamba install abricate
abricate --check
abricate --list
# download latest ABRicate database version
abricate-get_db --db ncbi --force